# Sensitivity data table to copy to latex file

Needs slight adaptation only.

In [1]:
%load_ext autoreload

In [2]:
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import xarray as xr
from tqdm import tqdm
from pathlib import Path
import pandas as pd

from LAPM.discrete_linear_autonomous_pool_model import DiscreteLinearAutonomousPoolModel as DLAPM
from CompartmentalSystems.discrete_model_run import DiscreteModelRun as DMR
import CompartmentalSystems.helpers_reservoir as hr

from BFCPM import utils
from BFCPM import DATA_PATH, FIGS_PATH, Q_, zeta_dw
from BFCPM.trees.single_tree_allocation import allometries
from BFCPM.trees.single_tree_params import species_params

%autoreload 2

In [3]:
# set plotting properties

mpl.rcParams['lines.linewidth'] = 2

SMALL_SIZE = 16
MEDIUM_SIZE = 17
#BIGGER_SIZE = 30

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
#plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## Load available simulations and sort them

In [4]:
all_sims_path = DATA_PATH.joinpath("simulations")

#sim_date = "2023-05-20" # differenct emergency strategies
#sim_date = "2023-06-08" # corrected wood density
#sim_date = "2023-06-19" # automatic thinning stand to SBA=18 on emergency cutting
#sim_date = "2023-07-26" # publication


spinup_length = 160

#sim_cohort_path = all_sims_path.joinpath(sim_date)
sim_cohort_path = all_sims_path.joinpath("sensitivity_full_sim")

print(sim_cohort_path)

/mnt/c/Users/hrme0001/GitHub/BFCPM/data/simulations/sensitivity_full_sim


## Check if the simulations have a proper continuous-cover spinup
This means, all trees survived and where cut only following the schedule.

In [5]:
def check_proper_cc_spinup(ds):
    proper_years = {
        0: [20, 100],
        1: [40, 120],
        2: [60, 140],
        3: [80, 160]
    }

    tree_nrs, yrs = np.where(ds.thinning_or_cutting_tree.data[:, :4]==1)
    for tree_nr, yr in zip(tree_nrs, yrs):
        if yr <= 160:
            if yr not in check_proper_cc_spinup[tree_nr]:
                return False

    return True

In [6]:
def get_par_name_and_q(s):
    par_names = ["rho_RL", "R_mL", "S_R", "Vcmax"]
    for par_name in par_names:
        if s.find(par_name) == -1:
            continue
    
        q_s = s.split(par_name + "_")[1]
        q = float(q_s[0] + "." + q_s[1:])

        return par_name, q

In [7]:
# reconcile filenames and simulation names
eligible_sim_names = {
    "mixed-aged_pine_N1500": "mixed-aged_pine",
    "even-aged_pine": "even-aged_pine",
    "even-aged_spruce": "even-aged_spruce",
    "even-aged_mixed": "even-aged_mixed",
}

In [8]:
col_names = [
    "sim_name", "proper_cc_spinup", "par_name", "q",
    "total_C",
    "WP_cum", "WPS_cum", "WPL_cum",
    "INCB", "IITT", "ICS"
]
d = {col_name: list() for col_name in col_names}

for p in sim_cohort_path.glob("**/*.nc"):
    if p.stem.find("_long") != -1:
#        print(p.stem, p.parent.parent.name)
        ds_long = xr.load_dataset(str(p))

        file_sim_name = p.stem.replace("_long", "")
        d["sim_name"].append(eligible_sim_names[file_sim_name])

        d["proper_cc_spinup"].append(check_proper_cc_spinup(ds_long))
        par_name, q = get_par_name_and_q(p.parent.parent.name)
        d["par_name"].append(par_name)
        d["q"].append(q)

        ds = xr.open_dataset(str(p.parent.joinpath(file_sim_name + ".nc")))

        d["total_C"].append(ds.stocks.sum(dim=["entity", "pool"]).data[-1] * 1e-03)
        
        d["WPS_cum"].append(ds.internal_fluxes.sel(pool_to="WP_S").sum(dim=["entity_to", "entity_from", "pool_from"]).cumsum(dim="time").data[-1] * 1e-03)
        d["WPL_cum"].append(ds.internal_fluxes.sel(pool_to="WP_L").sum(dim=["entity_to", "entity_from", "pool_from"]).cumsum(dim="time").data[-1] * 1e-03)
        d["WP_cum"].append(d["WPS_cum"][-1] + d["WPL_cum"][-1])

        x0 = ds.stocks.isel(time=0).sum(dim=["entity", "pool"])
        d["INCB"].append((ds.stocks.sum(dim=["entity", "pool"]) - x0).data[-1] * 1e-03)
        d["IITT"].append(ds.CS_through_time.data[-1] * 1e-03)
        d["ICS"].append(ds.stocks.sum(dim=["entity", "pool"]).cumsum(dim="time").data[-1] * 1e-03)

df = pd.DataFrame(d)
df

,sim_name,proper_cc_spinup,par_name,q,total_C,WP_cum,WPS_cum,WPL_cum,INCB,IITT,ICS
0,even-aged_mixed,True,rho_RL,0.90,18.179583,12.301775,3.950281,8.351494,2.920101,610.352189,1124.727044
1,even-aged_pine,True,rho_RL,0.90,17.378792,12.798230,4.845003,7.953227,2.119309,544.381838,1058.752673
2,even-aged_spruce,True,rho_RL,0.90,11.647721,8.868406,2.448330,6.420077,-3.611761,550.488182,1064.839364
3,mixed-aged_pine,True,rho_RL,0.90,14.479015,14.275952,5.102167,9.173785,-0.780467,571.099258,1109.353908
4,even-aged_mixed,True,rho_RL,0.95,18.423886,11.927509,3.981542,7.945966,3.573496,607.159892,1105.365336
...,...,...,...,...,...,...,...,...,...,...,...
75,mixed-aged_pine,True,Vcmax,1.05,14.475400,14.075392,5.156448,8.918944,-0.809911,571.967647,1109.646986
76,even-aged_mixed,True,Vcmax,1.10,18.349777,12.002955,3.933428,8.069526,2.523069,632.165377,1168.137716
77,even-aged_pine,True,Vcmax,1.10,17.922518,12.592999,4.886915,7.706084,2.095811,559.517422,1095.485354
78,even-aged_spruce,True,Vcmax,1.10,15.570738,9.884018,2.258768,7.625250,-0.255969,600.634306,1136.582945


In [9]:
sim_names = ["mixed-aged_pine", "even-aged_pine", "even-aged_spruce", "even-aged_mixed"]
df.sort_values(by="sim_name", key=lambda xs: [sim_names.index(x) for x in xs]).reset_index(drop=True)

,sim_name,proper_cc_spinup,par_name,q,total_C,WP_cum,WPS_cum,WPL_cum,INCB,IITT,ICS
0,mixed-aged_pine,True,R_mL,1.10,13.203859,13.026213,5.472241,7.553972,-0.822917,525.094035,1012.268310
1,mixed-aged_pine,True,Vcmax,1.05,14.475400,14.075392,5.156448,8.918944,-0.809911,571.967647,1109.646986
2,mixed-aged_pine,True,Vcmax,1.00,13.847517,13.568066,5.310351,8.257715,-0.808100,548.971758,1061.716850
3,mixed-aged_pine,True,Vcmax,0.95,13.121547,12.979498,5.499864,7.479635,-0.807826,522.692534,1006.436521
4,mixed-aged_pine,True,Vcmax,0.90,12.274400,12.300841,5.724141,6.576700,-0.776807,492.770429,940.959140
...,...,...,...,...,...,...,...,...,...,...,...
75,even-aged_mixed,True,rho_RL,1.05,17.770537,11.215832,4.054560,7.161272,3.724283,593.904939,1060.198452
76,even-aged_mixed,True,R_mL,0.95,18.281784,11.603152,4.011012,7.592140,3.306487,609.078103,1111.282684
77,even-aged_mixed,True,R_mL,0.90,18.619694,11.756682,4.006891,7.749792,3.320730,619.121304,1133.840701
78,even-aged_mixed,True,Vcmax,0.95,17.308421,11.031858,4.059348,6.972510,3.379049,579.783398,1040.982142


In [10]:
def SEM(x):
    return x.sem()

def RS(x):
    return np.ptp(x) / np.mean(x) * 100

In [11]:
pd.options.display.float_format = '{:,.2f}'.format
df_sens = df.groupby(by=["par_name", "sim_name"])[["total_C", "WP_cum", "IITT", "ICS"]].agg(RS)#.reset_index()
df_sens = df_sens.reindex(sim_names, level="sim_name")
#df_sens.columns = ["Parameter", "Scenario", "total carbon", "cum. WP", "IITT", "ICS"]
df_sens#.columns

total_C  WP_cum  IITT   ICS
par_name sim_name                                     
R_mL     mixed-aged_pine      9.39    8.07  8.81  9.41
         even-aged_pine       4.52    6.40  4.17  6.97
         even-aged_spruce     8.24   11.43  6.88  8.32
         even-aged_mixed      9.85    6.17  6.50  8.09
S_R      mixed-aged_pine      6.82    7.42  6.21  6.77
         even-aged_pine       5.25    8.55  1.91  4.37
         even-aged_spruce    30.36    6.96  5.04  3.01
         even-aged_mixed      2.81    6.96  1.45  3.99
Vcmax    mixed-aged_pine     19.95   16.41 18.19 19.93
         even-aged_pine       9.74   13.66 12.37 17.27
         even-aged_spruce    11.85   18.15 11.47 16.58
         even-aged_mixed     10.10   12.69 12.64 17.12
rho_RL   mixed-aged_pine     12.29   12.31 11.29 12.26
         even-aged_pine       4.84   13.11  4.79  8.78
         even-aged_spruce    32.40    4.09  4.96  4.66
         even-aged_mixed      5.69   12.21  3.81  7.99

In [12]:
s = df_sens.to_latex(float_format=lambda x: '%10.2f' % x)
s = s.replace("par_name", r"\thead{Parameter}").replace("sim_name", r"\thead{Scenario}")
s = s.replace("\\toprule", "").replace("\\midrule", "").replace("\\bottomrule", "")
s = s.replace("total_C", r"\thead{total stock}").replace("WP_cum", r"\thead{cum. WP}")
s = s.replace("IITT", r"\thead{IITT}").replace("ICS", r"\thead{ICS}")
s = s.replace("R_mL", r"$R_{\text{mL}}$").replace("S_R", r"$S_R$").replace("Vcmax", r"$V_{cmax,25}$").replace("rho_RL", r"$\rho_{\text{RL}}$")
s = s.replace("aged_", "aged ")

s = s.replace("mixed-aged pine", r"mixed-aged pine${}^1$", 1)
s = s.replace("even-aged pine", r"mixed-aged pine${}^2$", 1)
s = s.replace("even-aged spruce", r"mixed-aged pine${}^3$", 1)
s = s.replace("even-aged mixed", r"mixed-aged pine${}^4$", 1)

s = s.replace("19.95", r"19.95${}^1$")
s = s.replace("17.27", r"17.27${}^2$")
s = s.replace("32.40", r"32.40${}^3$")
s = s.replace("12.69", r"12.69${}^4$")

s = s.replace("11.43", r"\textbf{11.43}").replace("30.36", r"\textbf{30.36}").replace("19.95", r"\textbf{19.95}").replace("32.40", r"\textit{\textbf{32.40}}")
s = s.replace("18.15", r"\textit{18.15}").replace("18.19", r"\textit{18.19}").replace("19.93", r"\textit{19.93}")

print(s)

\begin{tabular}{llrrrr}

 &  & \thead{total stock} & \thead{cum. WP} & \thead{IITT} & \thead{ICS} \\
\thead{Parameter} & \thead{Scenario} &  &  &  &  \\

\multirow[t]{4}{*}{$R_{\text{mL}}$} & mixed-aged pine${}^1$ &       9.39 &       8.07 &       8.81 &       9.41 \\
 & mixed-aged pine${}^2$ &       4.52 &       6.40 &       4.17 &       6.97 \\
 & mixed-aged pine${}^3$ &       8.24 &      \textbf{11.43} &       6.88 &       8.32 \\
 & mixed-aged pine${}^4$ &       9.85 &       6.17 &       6.50 &       8.09 \\
\cline{1-6}
\multirow[t]{4}{*}{$S_R$} & mixed-aged pine &       6.82 &       7.42 &       6.21 &       6.77 \\
 & even-aged pine &       5.25 &       8.55 &       1.91 &       4.37 \\
 & even-aged spruce &      \textbf{30.36} &       6.96 &       5.04 &       3.01 \\
 & even-aged mixed &       2.81 &       6.96 &       1.45 &       3.99 \\
\cline{1-6}
\multirow[t]{4}{*}{$V_{cmax,25}$} & mixed-aged pine &      \textbf{19.95}${}^1$ &      16.41 &      \textit{18.19} &      \texti